In [ ]:
from latentis.space import LatentSpace
from latentis.sample import Uniform
from latentis.transform import projection
from latentis.transform.projection import RelativeProjection
import torch

In [ ]:
space = LatentSpace(vector_source=torch.randn(5, 10))
anchors = space.sample(sampler=Uniform(), n=3).vectors
space = space.vectors
space.shape, anchors.shape

In [ ]:
from latentis.transform.base import Centering


projector = RelativeProjection(projection_fn=projection.cosine_proj, abs_transform=Centering())
# Object API
rel_space1 = projector.fit(x=anchors).transform(x=space)[0]
rel_space2 = projector.fit(x=anchors)(x=space)["x"]
assert torch.allclose(rel_space1, rel_space2)
rel_space1

In [ ]:
# Functional API
from latentis.transform.functional import centering_state, centering_transform

centering_params = centering_state(x=anchors)
centered_space = centering_transform(x=space, **centering_params)
centered_anchors = centering_transform(x=anchors, **centering_params)
rel_space3 = projection.cosine_proj(x=centered_space, anchors=centered_anchors)
rel_space4 = projection.cosine_proj(x=space - anchors.mean(dim=0), anchors=anchors - anchors.mean(dim=0))
assert torch.allclose(rel_space1, rel_space3)
assert torch.allclose(rel_space1, rel_space4)